#### Importing packages

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.regularizers import l2
from sklearn.metrics import accuracy_score
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_validate

#### Upload dataset

In [6]:
data = pd.read_csv('../raw_data/dataset_phishing.csv')
data.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


#### Preprocessing data

##### Remove corrupted records

In [7]:
data.drop(data[data['domain_age'] <= -1].index, inplace=True) # remove corrupted records
data['domain_age'].describe()

count     9593.000000
mean      4840.694048
std       2782.092889
min          0.000000
25%       2429.000000
50%       5076.000000
75%       7287.000000
max      12874.000000
Name: domain_age, dtype: float64

##### Drop irrelevant columns

In [8]:
data.drop(columns=['statistical_report', 'phish_hints', 
                   'domain_in_brand', 'brand_in_subdomain', 'brand_in_path', 
                   'web_traffic', 'page_rank', 'dns_record'], inplace=True)

##### Encode target

In [9]:
class_map = {'legitimate':0, 'phishing':1}
data['status'] = data['status'].map(class_map)

#### Define X and y

In [15]:
X = data.drop(columns=['url', 'status'])
y = data['status']

#### Split data on train and test sets

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=42)

#### Scale train and test data

In [17]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Initialize the model

In [72]:
def initialize_model():
    model = Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(79,)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.00035)
    model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    return model

#### Fitting the model

In [31]:
X_train.shape, y_train.shape

((6715, 79), (6715,))

In [73]:
model = initialize_model()

es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, 
                    epochs=100, batch_size=32, 
                    verbose=1, callbacks=[es], 
                    validation_data=(X_test, y_test))

Epoch 1/100
210/210 [==============================] - 0s 738us/step - loss: 149.7721 - accuracy: 0.5148 - val_loss: 30.7070 - val_accuracy: 0.5730
Epoch 2/100
210/210 [==============================] - 0s 454us/step - loss: 68.0513 - accuracy: 0.5273 - val_loss: 10.3403 - val_accuracy: 0.6063
Epoch 3/100
210/210 [==============================] - 0s 450us/step - loss: 42.3145 - accuracy: 0.5631 - val_loss: 17.0254 - val_accuracy: 0.6081
Epoch 4/100
210/210 [==============================] - 0s 453us/step - loss: 29.0781 - accuracy: 0.6110 - val_loss: 7.2597 - val_accuracy: 0.7192
Epoch 5/100
210/210 [==============================] - 0s 464us/step - loss: 21.8064 - accuracy: 0.6287 - val_loss: 4.6380 - val_accuracy: 0.7561
Epoch 6/100
210/210 [==============================] - 0s 462us/step - loss: 16.0063 - accuracy: 0.6597 - val_loss: 4.6524 - val_accuracy: 0.7787
Epoch 7/100
210/210 [==============================] - 0s 459us/step - loss: 11.2728 - accuracy: 0.6873 - val_loss: 2.65

#### Testing the model

In [75]:
test_accuracy = model.evaluate(X_test, y_test)[1]
test_accuracy

90/90 [==============================] - 0s 265us/step - loss: 0.2353 - accuracy: 0.9124


0.9124391674995422

#### Save the model

In [76]:
from tensorflow.keras.models import load_model, save_model

save_model(model, f'../saved_models/model_{int(round(test_accuracy, 2) * 100)}.h5')